In [2]:
import pandas as pd
import numpy as np
import re
import tweepy
import json
from textblob import TextBlob
pd.set_option('display.max_colwidth', 280)
import time
from datetime import timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
# nlp.add_pipe('spacytextblob')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package vader_lexicon to C:\Users\Intel PC
[nltk_data]     G5900\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [36]:
vader.polarity_scores('I am great!')

{'neg': 0.0, 'neu': 0.185, 'pos': 0.815, 'compound': 0.6588}

In [6]:
eng_twits = pd.read_csv('eng_twits.csv')
eng_twits['tweet_id'] = eng_twits['tweet_id'].apply(lambda x: x if pd.isnull(x) else str(x))
eng_twits['user_id'] = eng_twits['user_id'].apply(lambda x: x if pd.isnull(x) else str(x))

In [7]:
eng_twits.shape

(1450, 13)

In [111]:
def cleantxt(text):
    '''
    Creat a function wich will remove mentions, remove hashtag words, removing the # symbol
    removing RT, removing the hyper link and other unnecessary characters.
    '''
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) #remove mentions
    text = re.sub(r"#(\w+)", ' ', text) #remove hashtag words
    text = re.sub(r'#', ' ', text) #removing the # symbol
    text = re.sub(r'RT[\s]+', ' ', text) #removing RT
    text = re.sub(r'https?:\/\/\S+', ' ', text) #removing the hyper link
    text = text.replace('\n', ' ')
    text = text.replace('&amp;', ' ')
    text = text.replace('&gt;', ' ')
    text = text.replace('_', ' ')
    text = text.strip()
#     text = text.lower()
    return text

def remove_emojis(data):
    '''
    Remove alla emoticons, flags, symbols and other characters.
    '''
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def polarity(txt):
    '''
    Compute from Textblob text's polarity.
    '''
    twit_polarity = TextBlob(txt).sentiment.polarity
    return twit_polarity

def subjectivity(txt):
    '''
    Compute from Textblob text's subjectivity.
    '''
    twit_subjectivity = TextBlob(txt).sentiment.subjectivity
    return twit_subjectivity

def polarity_kind(num):
    '''
    Distinguish tweets in Positives, Neutrals and Negatives from Textblob's polarity.
    '''
    if num > 0.15:
        polarity_kind = 'Positive'
    elif num < 0:
        polarity_kind = 'Negative'
    else:
        polarity_kind = 'Neutral'
    return polarity_kind

def vader_polarity(txt):
    '''
    Compute from Vader text's polarity.
    '''
    vader_polarity = vader.polarity_scores(txt)['compound']
    return vader_polarity

def vader_polarity_kind(num):
    '''
    Distinguish tweets in Positives, Neutrals and Negatives from Vader's polarity.
    '''
    if num > 0.15:
        vader_polarity_kind = 'Positive'
    elif num < -0.15:
        vader_polarity_kind = 'Negative'
    else:
        vader_polarity_kind = 'Neutral'
    
    return vader_polarity_kind

In [112]:
eng_twits['cleaned_tweets'] = eng_twits['tweets'].apply(lambda x: x if pd.isnull(x) else cleantxt(x))
eng_twits['noemojis_tweets'] = eng_twits['cleaned_tweets'].apply(lambda x: x if pd.isnull(x) else remove_emojis(x))
eng_twits['twits_polarity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else polarity(x))
eng_twits['twits_subjectivity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else subjectivity(x))
eng_twits['polarity_kinds'] = eng_twits['twits_polarity'].apply(lambda x: x if pd.isnull(x) else polarity_kind(x))
eng_twits['twits_vader_polarity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else vader_polarity(x))
eng_twits['vader_polarity_kinds'] = eng_twits['twits_vader_polarity'].apply(lambda x: x if pd.isnull(x) else vader_polarity_kind(x))


In [120]:
eng_twits[eng_twits['polarity_kinds'] == 'Positive'][['noemojis_tweets','twits_polarity', 'twits_subjectivity', 'polarity_kinds']].head(5)

,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds
1,I'm just gonna leave this here. Evacuation of . Straight out of a horror movie.,0.200000,0.400000,Positive
10,She ain't a famous influencer nor a tiktok star. Her name's Katerina Ioannidou she's a volunteer forest firefighter. Marios Lolos,0.500000,1.000000,Positive
11,"In Varimpompi, Greece, citizens are rounding up even at this time at night to help deliver food, drinks, and other important stuff to deliver to firefighters and fire-stricken citizens in Malakasa, Evoia and many more.",0.318750,0.593750,Positive
12,In case you don’t know what’s happening in Greece right now:,0.285714,0.535714,Positive
14,"In Varimpompi, Greece, citizens are rounding up even at this time at night to help deliver food, drinks, and other important stuff to deliver to firefighters and fire-stricken citizens in Malakasa, Evoia and many more.",0.318750,0.593750,Positive


In [121]:
eng_twits[eng_twits['polarity_kinds'] == 'Neutral'][['noemojis_tweets','twits_polarity', 'twits_subjectivity', 'polarity_kinds']].head(5)

,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds
0,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.",0.133333,0.56,Neutral
3,"Y'all dreaming about vacations in Greece posting pretty photos.. let me show you a ""pretty photo"". This was captured by a Greek photographer on the national road of Lamia - Athens. It translates to: 'Greece, how sorry I am.' We're living a holocaust..",0.000000,0.75,Neutral
4,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.",0.133333,0.56,Neutral
6,HEROES,0.000000,0.00,Neutral
7,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.",0.133333,0.56,Neutral


In [122]:
eng_twits[eng_twits['polarity_kinds'] == 'Negative'][['noemojis_tweets','twits_polarity', 'twits_subjectivity', 'polarity_kinds']].head(5)

,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds
2,"RT!! please. i’m begging my country is being burned to the ground. i’m chronically ill disabled, take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com",-0.066667,0.600000,Negative
5,"RT!! please. i’m begging my country is being burned to the ground. i’m chronically ill disabled, take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com",-0.066667,0.600000,Negative
24,"This is not ONLY climate change. has had similar EVERY YEAR for as long as I can remember. What we need is to start electing governments that actually care for nature, the place, the people, animals, etc. for a change, and not their pockets ego.",-0.012500,0.475000,Negative
30,"RT!! please. i’m begging my country is being burned to the ground. i’m chronically ill disabled, take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com",-0.066667,0.600000,Negative
34,"The largest active wildfire in Europe the Mediterranean is currently located in North Èvoia. 1.100 homes have been burned in less than a week. A tragedy, the worst natural catastrophe in modern Greece and it hasn't stopped yet.",-0.166667,0.461111,Negative


In [114]:
eng_twits['polarity_kinds'].value_counts()

Neutral     780
Positive    410
Negative    178
Name: polarity_kinds, dtype: int64

In [115]:
eng_twits['vader_polarity_kinds'].value_counts()

Positive    824
Negative    286
Neutral     258
Name: vader_polarity_kinds, dtype: int64

In [127]:
print("Textblob's mean polarity: ", eng_twits['twits_polarity'].mean())
print("Textblob's mean subjectivity: ", eng_twits['twits_subjectivity'].mean(), '\n')
print("Vader's mean polarity: ", eng_twits['twits_vader_polarity'].mean())

Textblob's mean polarity:  0.12970357522845583
Textblob's mean subjectivity:  0.47136775904909645 

Vader's mean polarity:  0.2972186403508773


In [128]:
eng_twits.to_csv('eng_twits_analyzed.csv', index=False)

In [75]:
eng_twits_analyzed = pd.read_csv('eng_twits_analyzed.csv')